In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics


import pandas as pd
import numpy as np

from fractions import Fraction as frac

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("fa21-ds5110-group10-rk") \
    .config("spark.driver.memory", "16g") \
    .getOrCreate()

First we add in our cached dataset from our prior feature engineering.

In [3]:
df = spark.read.parquet("../../../data/processed/chess_games_moves_model.parquet")

In [4]:
df.cache()

DataFrame[event: string, white_result: string, first_two: array<string>, ECO: string, EloDiff: int, game_complexity: int]

In [5]:
df.show(2)

+-----+------------+----------------+---+-------+---------------+
|event|white_result|       first_two|ECO|EloDiff|game_complexity|
+-----+------------+----------------+---+-------+---------------+
|Blitz|         win|[c4 c5, Nc3 Nf6]|A34|    222|              6|
|Blitz|         win| [d4 b6, c4 Bb7]|A40|    309|              5|
+-----+------------+----------------+---+-------+---------------+
only showing top 2 rows



Next we review the resulting data points of interest.
We notice that ECO and the first two sets of moves are distinct of one another, and may influence the overall model's prediction.

In [6]:
print("First Turns: {}".format(df.select("first_two").distinct().count()))
print("ECO Types: {}".format(df.select("ECO").distinct().count()))
print( df.groupBy('first_two').count().sort(F.col("count").desc()).show(5), df.count())

First Turns: 464628
ECO Types: 493
+------------------+------+
|         first_two| count|
+------------------+------+
|  [e4 e5, Nf3 Nc6]|332412|
|   [e4 e5, Nf3 d6]|129785|
|[e4 d5, exd5 Qxd5]| 86575|
|  [e4 c5, Nf3 Nc6]| 78638|
|    [e4 e6, d4 d5]| 72122|
+------------------+------+
only showing top 5 rows

None 3849646


We will now begin to build a model, keying in on the opening move and the white_result columns.
Note that both of these are categorical values, so we will need to encode them using the StringIndexer for pyspark to do model evaluations.

In [7]:
# features
opening_vectorizor = StringIndexer(inputCol="ECO", outputCol="opening_ohe")
gametype_vectorizer = StringIndexer(inputCol="event", outputCol="event_vector")
# target
result_vectorizor = StringIndexer(inputCol="white_result", outputCol="white_result_vector")

Next we perform One-Hot Encoding on our Opening type (or ECO) and do our comparision.  THis will create a new column that we will use for our random forest model.

In [8]:
move_encoder = OneHotEncoder(inputCols=["opening_ohe"],
                        outputCols=["ECO_Type"])

Now that we have the OHE of our ECO, we can combine it with other features to build out our predictors for random forest.

In [9]:
features_assembler = VectorAssembler(inputCols=['ECO_Type', "event_vector","EloDiff"], outputCol='features')

Now that our data is model-ready, we will do a split, fit, transform, and evaluation to determine the performance of our model.
Note that we have chosen the default tunings, but in the future we will likely apply a cross-validation technique in pyspark to select the correct hyperparameters.

In [10]:
trainData, testData, validationData  = df.randomSplit([0.5, 0.4, 0.1], seed=314)

In [11]:
gbt = GBTClassifier(maxDepth=5, maxIter=5, labelCol='white_result_vector', seed=1337, leafCol="leafId")

In [12]:
pipeline = Pipeline(stages=[opening_vectorizor,
                            gametype_vectorizer,
                            result_vectorizor,
                            move_encoder,
                            features_assembler,
                            gbt])

In [13]:
model = pipeline.fit(validationData)

In [14]:
result = model.transform(testData)

In [15]:
result.show(2)

+-----+------------+-----------------+---+-------+---------------+-----------+------------+-------------------+---------------+--------------------+--------------------+--------------------+----------+--------------------+
|event|white_result|        first_two|ECO|EloDiff|game_complexity|opening_ohe|event_vector|white_result_vector|       ECO_Type|            features|       rawPrediction|         probability|prediction|              leafId|
+-----+------------+-----------------+---+-------+---------------+-----------+------------+-------------------+---------------+--------------------+--------------------+--------------------+----------+--------------------+
|Blitz|        loss|[Na3 Nf6, Nf3 d5]|A00|     80|              5|        0.0|         0.0|                0.0|(469,[0],[1.0])|(471,[0,470],[1.0...|[-0.1676323757136...|[0.41696018215025...|       1.0|[21.0,17.0,18.0,1...|
|Blitz|        loss|  [Na3 d5, b3 c5]|A00|     21|              5|        0.0|         0.0|                0

In [16]:
accuracy_evaluator = BinaryClassificationEvaluator(labelCol='white_result_vector',rawPredictionCol="prediction")
model_accuracy = accuracy_evaluator.evaluate(result)

Py4JJavaError: An error occurred while calling o485.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 73.0 failed 1 times, most recent failure: Lost task 3.0 in stage 73.0 (TID 850) (udc-ba33-13c8 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$3245/0x0000000841409840: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at or

In [ ]:
print('Accuracy:', gbevaluator.evaluate(gbpredictions))
print('AUC:', BinaryClassificationMetrics(gbpredictions['label','prediction'].rdd).areaUnderROC)
print('PR:', BinaryClassificationMetrics(gbpredictions['label','prediction'].rdd).areaUnderPR)

In [ ]:
print("Model Accuracy: {}".format(model_accuracy))

Here we see that our model's performance seems to work well, with roughtly 50% of games resulting in a match to white win.

Let's review our classifications and confusion matrix next to determine the overall performance.

In [ ]:
designer_matrix = rf_result.select(['prediction','white_result_vector'])
metrics_rdd = MulticlassMetrics(designer_matrix.rdd.map(tuple))
print(metrics_rdd.confusionMatrix().toArray())

Reviewing this model, we see that this model is not good.
Of the three classes, we found that the precision for the white loss performs okay, however the white win and tie both evaluate to 0 for precision.
This means that our model is overfitting and failed to correctly identify either ties or losses.

Further tuning will be required to better distribute our data (either adjusting the threshold, or tuning the tree).